# Project: 12-2023 Instacart Basket Analysis
## Author: Nadia Ordonez
## Step 6 IC Derivations and Aggregations

# Table of Contents
* [1. Introduction](#1.-Introduction)
* [2. Importing data](#2.-Importing-data)
    * [2.1 Importing libraries](#2.1-Importing-libraries)
    * [2.2 Importing data](#2.2-Importing-data)
* [3. Price range products](#3.-Price-range-products)
* [4. Busiest days](#4.-Busiest-days)
* [5. Busiest period of day](#5.-Busiest-period-of-day)
* [6. Region](#6.-Region)
* [7. Loyalty flag](#7.-Loyalty-flag)
* [8. Type of spender](#8.-Type-of-spender)
* [9. Usage frequency](#9.-Usage-frequency)
* [10. Exporting data](#10.-Exporting-data) 

# 1. Introduction

* To answer Instacart research question, all dataframes were combined into a single dataframe that excludes customers with less than 5 placed orders as requested by CFO. Here, aggregations and derived variables will be created to enrich our analyses as requested by different stakeholders.
* These variables will be created out of existing variables in our dataframe:
    * "price_range_products" containing labels for high, middle and low range products.
    * "busiest_days" containing labels busiest, regular and least busiest days.
    * "busiest_period_of_day" containing labels most, average, fewest orders.
    * "region" grouping states into regions.
    * "loyalty_flag" containing labels loyal, regular, new customer.
* These variables will be created after calculating an aggregate variable from an existing variable:
    * "type_of_spender" containing labels high and low spenders.
    * "usage_frequency" containing labels frequent, regular and non-frequent. 

# 2. Importing data

## 2.1 Importing libraries

In [1]:
#Import analytical libraries
import pandas as pd
import numpy as np
import os

## 2.2 Importing data

In [2]:
#Project folder path into a string to easily retrieve data
path = r'C:\Users\Ich\Documents\12-2023 Instacart Basket Analysis'

### Order products final

In [3]:
#Import “orders_products_final_step5.pkl”
#See "Step 5 IC Orders products final" to check for merging details
orders_products_final = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared data', 'orders_products_final_step5.pkl'))

In [4]:
#Check df size
orders_products_final.shape

(30992966, 21)

In [5]:
#Check headers
orders_products_final.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,...,department_id,prices,gender,state,age,date_joined,number_of_dependants,family_status,income,max_order_number
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423,10
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423,10
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423,10
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423,10
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,7,9.0,Female,Alabama,31,2019-02-17,3,married,40423,10


# 3. Price range products

* Different marketing strategies are applied when selling a 2 dollar candy bar than a 15 dollar box of Belgian chocolates! It makes sense to categorize products into price range groups for easy filtering. The following conditions are applied to "prices" to create the variable "price_range_products":
    * If the item’s price is lower than or equal to 5 dollars, it will be labeled a “low-range”.
    * If the item’s price is above 5 dollars but lower than or equal to 15 dollars, it will be labeled a “mid-range”.
    * If the item’s price is above 15 dollars, it will be labeled a “high-range”.

## Variable evaluation

The "prices" variable would be used to create the "price_range_products" labels.

In [6]:
#Evaluate "prices"
orders_products_final['prices'].describe()

count    3.098809e+07
mean     7.795331e+00
std      4.241884e+00
min      1.000000e+00
25%      4.200000e+00
50%      7.400000e+00
75%      1.130000e+01
max      2.500000e+01
Name: prices, dtype: float64

In [7]:
#Check for missing values
#Missing values are expected (Step1 IC Data Import, Wrangling and Consistency checks)
orders_products_final['prices'].isnull().sum()
#NaN corresponds to two products that contained odd values (outliers) and were converted to NaN
#In Step 1, following deliverations with stakeholders, it was advised to convert odd price values to NaN

4877

## Conditions

In [8]:
orders_products_final.loc[orders_products_final['prices'] > 15, 'price_range_products'] = 'high-range'

C:\Users\Ich\AppData\Local\Temp\ipykernel_10992\1128871956.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'high-range' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  orders_products_final.loc[orders_products_final['prices'] > 15, 'price_range_products'] = 'high-range'


In [9]:
orders_products_final.loc[(orders_products_final['prices'] <= 15) & (orders_products_final['prices'] > 5), 'price_range_products'] = 'mid-range' 

In [10]:
orders_products_final.loc[orders_products_final['prices'] <= 5, 'price_range_products'] = 'low-range'

## Output evaluation

In [11]:
#Count values
orders_products_final['price_range_products'].value_counts(dropna = False)
#NaN corresponds to two items that contained odd values (outliers) that were converted to NaN
#For details see (Step1 IC Data Import, Wrangling and Consistency checks)
#In Step 1, following deliverations with stakeholders, it was advised to convert odd values to NaN

price_range_products
mid-range     20920614
low-range      9674395
high-range      393080
NaN               4877
Name: count, dtype: int64

In [12]:
#Check flags
orders_products_final.groupby('price_range_products').agg({'prices' : ['min', 'max']})
#nothing odd on the results

prices      
                        min   max
price_range_products             
high-range             15.1  25.0
low-range               1.0   5.0
mid-range               5.1  15.0

In [13]:
#Check results
orders_products_final[['prices', 'price_range_products']] .head(20)
#nothing odd on the results

,prices,price_range_products
0,9.0,mid-range
1,9.0,mid-range
2,9.0,mid-range
3,9.0,mid-range
4,9.0,mid-range
5,9.0,mid-range
6,9.0,mid-range
7,9.0,mid-range
8,9.0,mid-range
9,9.0,mid-range


The conditions were correctly assigned to all products EXCEPT for two products. NOTE: "price_range_products" contains NaN coming from two products (See Step 1 IC Data Import, Wrangling and Consistency checks).

# 4. Busiest days

* Our stakeholders want to create a “busiest_days” variable based on the frequency of orders in the "orders_day_of_week". The following conditions will be applied:
    * “Busiest days” corresponding to the two busiest days of the week. 
    * At the same time, our stakeholders also like to know the two slowest days, labeled as "Least busiest days".
    * The reamining days should be labeled as "Regular days".

## Variable evaluation

The variable "busiest_days" will be created based on the frequency of orders observed in the "orders_day_of_week" variable. After deliverations with stakeholders, it was agreed to evaluate the frequency of orders taking into account the volumen of orders per user, instead of single orders per user. In our current dataframe, a single order_id is repeated several items in the "order_id" variable depending on how many products were purchased within an order. 

In [14]:
#Evaluate "orders_day_of_week"
orders_products_final['orders_day_of_week'].describe()
#7 days of the week exists, starting at 0 as Saturday

count    3.099297e+07
mean     2.741350e+00
std      2.089228e+00
min      0.000000e+00
25%      1.000000e+00
50%      3.000000e+00
75%      5.000000e+00
max      6.000000e+00
Name: orders_day_of_week, dtype: float64

In [15]:
#Printing the frequency of a column will quickly inform you which values appear more often within that column.
orders_products_final['orders_day_of_week'].value_counts(dropna = False)
#the value 0 means Saturday
#0 and 1 are the "Busiest days"

orders_day_of_week
0    5913859
1    5417064
6    4294962
5    4037350
2    4028494
3    3672954
4    3628283
Name: count, dtype: int64

## Conditions

In [16]:
#The folllowing conditions are applied:
#If the value in "orders_day_of_week" equals to 0 or 1, a “Busiest days” string is appended to the result list. 
#If the value is equal to 4 and 3, a “Least busiest days” string is appended. 
#If neither of these conditions has been met (the value is neither 0, 1, 3 nor 4), then a “Regular days” is appended.

result = []

for value in orders_products_final["orders_day_of_week"]:
  if value == 0 or value == 1:
    result.append("Busiest days")
    #busiest days = 0 and 1 Saturday and Sunday
  elif value == 4 or value == 3:
    result.append("Least busiest days")
    #least busy days = 4 and 3 Wednesday and Tuesday
  else:
    result.append("Regular days")
    # 2, 5, and 6 Monday, Thrusday and Friday

In [17]:
#See results
result

['Regular days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Regular days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Least busiest days',
 'Regular days',
 'Regular days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Regular days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Least busiest days',
 'Regular days',
 'Least busiest days',
 'Regular days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Regular days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Regular days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Least busiest days',
 'Least busiest days',
 'Least busiest days',
 'Lea

In [18]:
#Adding the results to your dataframe
orders_products_final['busiest_days'] = result

## Output evaluation

In [19]:
#See results
orders_products_final.head()
#variable "busiest_days" was added

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,...,gender,state,age,date_joined,number_of_dependants,family_status,income,max_order_number,price_range_products,busiest_days
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Regular days
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Female,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days


In [20]:
#Counting values within new column
orders_products_final['busiest_days'].value_counts(dropna = False)

busiest_days
Regular days          12360806
Busiest days          11330923
Least busiest days     7301237
Name: count, dtype: int64

In [21]:
#Check flags
orders_products_final.groupby(['orders_day_of_week'])['busiest_days'].max()
#nothing odd on the results

orders_day_of_week
0          Busiest days
1          Busiest days
2          Regular days
3    Least busiest days
4    Least busiest days
5          Regular days
6          Regular days
Name: busiest_days, dtype: object

In [22]:
#Check results
orders_products_final[['orders_day_of_week', 'busiest_days']].head(20)
#nothing odd on the results

,orders_day_of_week,busiest_days
0,2,Regular days
1,3,Least busiest days
2,3,Least busiest days
3,4,Least busiest days
4,4,Least busiest days
5,2,Regular days
6,1,Busiest days
7,1,Busiest days
8,1,Busiest days
9,4,Least busiest days


The conditions were correctly assigned to all days of the week. Regular days are the most predominant days. 

# 5. Busiest period of day

* When too many users make Instacart orders at the same time, the app freezes. The senior technical officer at Instacart wants to identify the busiest hours of the day. Rather than by hour, they want periods of time labeled “Most orders” “Average orders” and “Fewest orders”. After deliverations with the senrior technical officer, the following conditions were used to create a “busiest_period_of_day” variable based on the frequency of orders in the "order_hour_of_day":
    * Since the app freezes usually when is processing more than 2.5M orders at a time, then hour of days with more than 2.5M will be labeled as "Most orders".
    * “Fewest orders" are those with < 100 thousand orders, since the app does not experience any problems at all with this volumen of orders.
    * The reamining hours of the day should be labeled as "Average orders".

## Variable evaluation

The variable "busiest_period_of_day" will be created based on the frequency of orders observed in the "order_hour_of_day" variable. After deliverations with the senior technical officer, it was agreed to evaluate the frequency of orders taking into account the volumen of orders per user, instead of single orders per user. In our current dataframe, a single order_id is repeated several items in the "order_id" variable depending on how many products were purchased within an order. 

In [23]:
#Evaluate "orders_hour_of_day"
orders_products_final['order_hour_of_day'].describe()

count    3.099297e+07
mean     1.341081e+01
std      4.248825e+00
min      0.000000e+00
25%      1.000000e+01
50%      1.300000e+01
75%      1.600000e+01
max      2.300000e+01
Name: order_hour_of_day, dtype: float64

In [24]:
#Printing the frequency of a column will quickly inform you which values appear more often within that column
orders_products_final['order_hour_of_day'].value_counts(dropna = False)

order_hour_of_day
10    2652317
11    2617229
14    2570213
13    2541527
15    2538253
12    2497997
16    2415839
9     2366037
17    1984069
8     1660891
18    1552540
19    1194918
20     931746
7      864069
21     763813
22     607550
23     384838
6      281307
0      208214
1      109705
5       84514
2       65743
4       50802
3       48835
Name: count, dtype: int64

## Conditions

In [25]:
#The folllowing conditions are applied:
#Since the app usually freezes when is processing more than 2.5M orders at a time, 
#then hour of days with more than 2.5M will be labeled as "Most orders"
#“Fewest orders" are those with < 100 thousand orders. The app does not experience any problems with this volumen of orders
#The reamining hours of the day should be labeled as "Average orders".

result = []

for value in orders_products_final["order_hour_of_day"]:
  if value in [15, 13, 14, 11, 10]:
    result.append("Most orders")
  elif value in [5, 2, 4, 3]:
    result.append("Fewest orders")
  else:
    result.append("Average orders")

In [26]:
#See results
result

['Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Most orders

In [27]:
#Adding the results to the df
orders_products_final['busiest_period_of_day'] = result

## Output evaluation

In [28]:
#See results
orders_products_final.head()
#variable "busiest_days" was added

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,...,state,age,date_joined,number_of_dependants,family_status,income,max_order_number,price_range_products,busiest_days,busiest_period_of_day
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Regular days,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days,Average orders
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days,Average orders
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days,Average orders
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Alabama,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days,Most orders


In [29]:
#Counting values within new column
orders_products_final['busiest_period_of_day'].value_counts(dropna = False)

busiest_period_of_day
Average orders    17823533
Most orders       12919539
Fewest orders       249894
Name: count, dtype: int64

In [30]:
#Check flags
orders_products_final.groupby(['order_hour_of_day'])['busiest_period_of_day'].max()
#nothing odd on the results

order_hour_of_day
0     Average orders
1     Average orders
2      Fewest orders
3      Fewest orders
4      Fewest orders
5      Fewest orders
6     Average orders
7     Average orders
8     Average orders
9     Average orders
10       Most orders
11       Most orders
12    Average orders
13       Most orders
14       Most orders
15       Most orders
16    Average orders
17    Average orders
18    Average orders
19    Average orders
20    Average orders
21    Average orders
22    Average orders
23    Average orders
Name: busiest_period_of_day, dtype: object

In [31]:
#Check results
orders_products_final[['order_hour_of_day', 'busiest_period_of_day']].head(20)
#nothing odd on the results

,order_hour_of_day,busiest_period_of_day
0,8,Average orders
1,7,Average orders
2,12,Average orders
3,7,Average orders
4,15,Most orders
5,7,Average orders
6,9,Average orders
7,14,Most orders
8,16,Average orders
9,8,Average orders


The conditions were correctly assigned to all hours of the day. Regular days are the most predominant days. Most orders are placed between 10:00 to 15:00.

# 6. Region

* The Instacart officers are interested in comparing customer behavior in different geographic areas. Here, a regional segmentation of the data will be created.The “region” column will be based on the states present in the “state” column. The regional segmentation will be created following the criteria on this link:
    * List of regions of the United States (https://simple.wikipedia.org/wiki/List_of_regions_of_the_United_States). Only regions will be created, not divisions. A dictionary will be created to map states to regions.

## Variable evaluation

The variable "state" will be used to create the "region" variable. 

In [32]:
# Identify the number of unique values in 'state'
unique_states = orders_products_final['state'].nunique()
print(f"The number of unique values in 'state': {unique_states}")
#There are 51 states listed

The number of unique values in 'state': 51


In [33]:
#Printing names of unique states
unique_states_names = orders_products_final['state'].unique()
print("Names of unique states:")
for state_name in unique_states_names:
    print(state_name)

Names of unique states:
Alabama
Indiana
Louisiana
Montana
Oklahoma
Tennessee
Iowa
Ohio
Virginia
Connecticut
Oregon
South Dakota
Kentucky
Michigan
New Jersey
Hawaii
Minnesota
Illinois
Idaho
Kansas
North Carolina
Vermont
Delaware
Nebraska
Nevada
Arkansas
Georgia
District of Columbia
Utah
Alaska
Colorado
Florida
Rhode Island
Maine
South Carolina
Washington
Mississippi
California
Texas
Arizona
New Mexico
Maryland
Missouri
Wyoming
New Hampshire
Massachusetts
Pennsylvania
Wisconsin
West Virginia
North Dakota
New York


## Conditions

In [34]:
#Create a dictionary mapping each state to its corresponding region
state_to_region = {
    'Alabama': 'South',
    'Indiana': 'Midwest',
    'Louisiana': 'South',
    'Montana': 'West',
    'Oklahoma': 'South',
    'Tennessee': 'South',
    'Iowa': 'Midwest',
    'Ohio': 'Midwest',
    'Virginia': 'South',
    'Connecticut': 'Northeast',
    'Oregon': 'West',
    'South Dakota': 'Midwest',
    'Kentucky': 'South',
    'Michigan': 'Midwest',
    'New Jersey': 'Northeast',
    'Hawaii': 'West',
    'Minnesota': 'Midwest',
    'Illinois': 'Midwest',
    'Idaho': 'West',
    'Kansas': 'Midwest',
    'North Carolina': 'South',
    'Vermont': 'Northeast',
    'Delaware': 'South',
    'Nebraska': 'Midwest',
    'Nevada': 'West',
    'Arkansas': 'South',
    'Georgia': 'South',
    'District of Columbia': 'South',
    'Utah': 'West',
    'Alaska': 'West',
    'Colorado': 'West',
    'Florida': 'South',
    'Rhode Island': 'Northeast',
    'Maine': 'Northeast',
    'South Carolina': 'South',
    'Washington': 'West',
    'Mississippi': 'South',
    'California': 'West',
    'Texas': 'South',
    'Arizona': 'West',
    'New Mexico': 'West',
    'Maryland': 'South',
    'Missouri': 'Midwest',
    'Wyoming': 'West',
    'New Hampshire': 'Northeast',
    'Massachusetts': 'Northeast',
    'Pennsylvania': 'Northeast',
    'Wisconsin': 'Midwest',
    'West Virginia': 'Midwest',
    'North Dakota': 'Midwest',
    'New York': 'Northeast'
}

In [35]:
#Adding the 'region' column to the df based on the mapping
orders_products_final['region'] = orders_products_final['state'].map(state_to_region)

In [36]:
#See results
orders_products_final.head()
#"region" variable was added

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,...,age,date_joined,number_of_dependants,family_status,income,max_order_number,price_range_products,busiest_days,busiest_period_of_day,region
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,31,2019-02-17,3,married,40423,10,mid-range,Regular days,Average orders,South
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days,Average orders,South
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days,Average orders,South
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days,Average orders,South
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,31,2019-02-17,3,married,40423,10,mid-range,Least busiest days,Most orders,South


## Output evaluation

In [37]:
#Counting values within new column
orders_products_final['region'].value_counts(dropna = False)

region
South        9737281
West         7934470
Midwest      7851135
Northeast    5470080
Name: count, dtype: int64

In [38]:
#Check flags
orders_products_final.groupby(['state'])['region'].max()
#nothing odd on the results

C:\Users\Ich\AppData\Local\Temp\ipykernel_10992\2633492751.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  orders_products_final.groupby(['state'])['region'].max()


state
Alabama                     South
Alaska                       West
Arizona                      West
Arkansas                    South
California                   West
Colorado                     West
Connecticut             Northeast
Delaware                    South
District of Columbia        South
Florida                     South
Georgia                     South
Hawaii                       West
Idaho                        West
Illinois                  Midwest
Indiana                   Midwest
Iowa                      Midwest
Kansas                    Midwest
Kentucky                    South
Louisiana                   South
Maine                   Northeast
Maryland                    South
Massachusetts           Northeast
Michigan                  Midwest
Minnesota                 Midwest
Mississippi                 South
Missouri                  Midwest
Montana                      West
Nebraska                  Midwest
Nevada                       West
New Hamp

Using a dictionary, all states in our dataframe were correctly mapped to a region. Most of orders are coming from states in the South region.

## 7. Loyalty flag

* The business strategy team would like to employ some kind of bonus point program for loyal Instacart customers. These are customers that come back time and time again and use the service or buy the product often. The "max_order_number" variable will be used to create the "loyalty_flag" following these conditions:
    * If the maximum orders the user has made is over 40, then the customer will be labeled a “Loyal”.
    * If the maximum orders the user has made is over 10 but less than or equal to 40, then the customer will be labeled “Regular”.
    * If the maximum orders the user has made is less than or equal to 10, then the customer will be labeled “New”.

## Variable evaluation

The variable "loyalty_flag" will be created based on the values observed in the "max_order_number" variable. See Step 5 IC Final dataset after exclusion flag to find out details how the "max_order_number" variable was created out of an aggregation from the "order_number" variable.  

In [39]:
#Evaluate "max_order_number"
orders_products_final['max_order_number'].describe()
#The min of 5 orders was set in Step 5 IC Final dataset after exclusion flag
#CFO is not interested in data analysis coming from users that placed less than 5 orders

count    3.099297e+07
mean     3.442599e+01
std      2.489448e+01
min      5.000000e+00
25%      1.400000e+01
50%      2.800000e+01
75%      4.800000e+01
max      9.900000e+01
Name: max_order_number, dtype: float64

## Conditions

In [40]:
#If the maximum orders the user has made is over 40, then the customer will be labeled a “Loyal”
orders_products_final.loc[orders_products_final['max_order_number'] > 40, 'loyalty_flag'] = 'Loyal'

C:\Users\Ich\AppData\Local\Temp\ipykernel_10992\3252715431.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Loyal' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  orders_products_final.loc[orders_products_final['max_order_number'] > 40, 'loyalty_flag'] = 'Loyal'


In [41]:
#If the maximum orders the user has made is over 10 but less than or equal to 40, then the customer will be labeled “Regular”
orders_products_final.loc[(orders_products_final['max_order_number'] <= 40) & (orders_products_final['max_order_number'] > 10), 'loyalty_flag'] = 'Regular'

In [42]:
#If the maximum orders the user has made is less than or equal to 10, then the customer will be labeled “New”
orders_products_final.loc[orders_products_final['max_order_number'] <= 10, 'loyalty_flag'] = 'New'

## Output evaluation

In [43]:
#Print the frequency of “loyalty_flag” 
orders_products_final['loyalty_flag'].value_counts(dropna = False) 

loyalty_flag
Regular    15891172
Loyal      10293959
New         4807835
Name: count, dtype: int64

In [44]:
#Check flags
orders_products_final.groupby('loyalty_flag').agg({'max_order_number' : ['min', 'max']})
#nothing odd on the results

max_order_number    
                          min max
loyalty_flag                     
Loyal                      41  99
New                         5  10
Regular                    11  40

In [45]:
#See results
orders_products_final[['loyalty_flag', 'max_order_number']].head()
#variable is added

,loyalty_flag,max_order_number
0,New,10
1,New,10
2,New,10
3,New,10
4,New,10


In [46]:
#See results
orders_products_final[['loyalty_flag', 'max_order_number']].tail()
#variable is added

,loyalty_flag,max_order_number
32434480,Regular,26
32434481,Regular,26
32434482,Regular,26
32434483,Regular,26
32434484,Regular,26


The conditions were correctly assigned to all our users. Loyal users corresponds to 33.2 % of our total users.

# 8. Type of spender

* Instacart wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. A spending flag for each user will be created based on the average price across all their orders using the following criteria:
    * If the mean of the prices of products purchased by a user is lower than 10, then they will labeled “Low spender”.
    * If the mean of the prices of products purchased by a user is higher than or equal to 10, then they will be labeled “High spender”.

## Aggregate variable

To create the variable "type_of_spender" firstly, the mean of the prices of products purchased by a user should be calculated. Here, an aggregate variable based on the "prices" variable will be created to assign a mean under the "mean_price_per_user" variable. 

In [47]:
#Calculating the average price of products purchased per user = ("mean_price_per_user")
orders_products_final['mean_price_per_user'] = orders_products_final.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\Ich\AppData\Local\Temp\ipykernel_10992\1571174399.py:2: FutureWarning: The provided callable <function mean at 0x000001C92C736E80> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  orders_products_final['mean_price_per_user'] = orders_products_final.groupby(['user_id'])['prices'].transform(np.mean)


In [48]:
#See results
orders_products_final.head()
# "mean_price_per_user" was added

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,...,number_of_dependants,family_status,income,max_order_number,price_range_products,busiest_days,busiest_period_of_day,region,loyalty_flag,mean_price_per_user
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,3,married,40423,10,mid-range,Regular days,Average orders,South,New,6.367797
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,3,married,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,3,married,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,3,married,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,3,married,40423,10,mid-range,Least busiest days,Most orders,South,New,6.367797


In [49]:
#Evaluate "mean_price_per_user"
orders_products_final['mean_price_per_user'].describe()
#Average price per product purchased per user ranges from 1 to 20.11 dollars

count    3.099297e+07
mean     7.795333e+00
std      7.216319e-01
min      1.000000e+00
25%      7.389744e+00
50%      7.816998e+00
75%      8.228163e+00
max      2.011429e+01
Name: mean_price_per_user, dtype: float64

In [50]:
#Check aggregations
#Snap view of prices per user
orders_products_final.groupby('user_id').agg({'prices' : ['min', 'max', 'mean']})
#nothing odd on the results

prices                
           min   max      mean
user_id                       
1          1.0  14.0  6.367797
2          1.3  14.8  7.515897
3          1.3  14.4  8.197727
4          1.4  14.6  8.205556
7          1.1  23.2  7.466990
...        ...   ...       ...
206203     1.0  25.0  7.138655
206206     1.1  14.8  7.646667
206207     1.2  24.7  7.313453
206208     1.1  21.6  8.366617
206209     1.1  18.8  7.058915

[162633 rows x 3 columns]

## Conditions

In [51]:
#If the mean of the prices of products purchased by a user is higher than or equal to 10, then they will be labeled “High spender”
orders_products_final.loc[orders_products_final['mean_price_per_user'] >= 10, 'type_of_spender'] = 'High spender'

C:\Users\Ich\AppData\Local\Temp\ipykernel_10992\2405728393.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'High spender' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  orders_products_final.loc[orders_products_final['mean_price_per_user'] >= 10, 'type_of_spender'] = 'High spender'


In [52]:
#If the mean of the prices of products purchased by a user is lower than 10, then they will labeled “Low spender”
orders_products_final.loc[orders_products_final['mean_price_per_user'] < 10, 'type_of_spender'] = 'Low spender'

## Output evaluation

In [53]:
#Print the frequency of “type_of_spender” 
orders_products_final['type_of_spender'].value_counts(dropna = False) 

type_of_spender
Low spender     30894578
High spender       98388
Name: count, dtype: int64

In [67]:
#Check aggregations
#Snap view of prices per user
orders_products_final.groupby('type_of_spender').agg({'mean_price_per_user' : ['min', 'max', 'mean']})
#nothing odd on the results

mean_price_per_user                      
                                min        max       mean
type_of_spender                                          
High spender                   10.0  20.114286  10.563224
Low spender                     1.0   9.998305   7.786518

In [55]:
#See results
orders_products_final.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,...,family_status,income,max_order_number,price_range_products,busiest_days,busiest_period_of_day,region,loyalty_flag,mean_price_per_user,type_of_spender
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,married,40423,10,mid-range,Regular days,Average orders,South,New,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,married,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,married,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,married,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,married,40423,10,mid-range,Least busiest days,Most orders,South,New,6.367797,Low spender


Most of our users (99.68%) are of "Low spender" type. 

# 9. Usage frequency

* In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants to determine frequent versus non-frequent customers. It was agreed that this frequency flag that marks the regularity of a user’s ordering behavior will be based on the median in the “days_since_prior_order” variable. The criteria for the flag should be as follows:
    * If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer”.
    * If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer”.
    * If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer”.

## Aggregate variable

To create the variable "usage_frequency" firstly, the median of the "days_since_prior_order" variable per user should be calculated. Here, an aggregate variable based on the "days_since_prior_order" variable will be created to assign a media per user under the "median_days_since_prior_order" variable.

In [56]:
#Calculating the median per user = ("median_days_since_prior_order") that each user took before placing a follow-up order
orders_products_final['median_days_since_prior_order'] = orders_products_final.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\Ich\AppData\Local\Temp\ipykernel_10992\649786976.py:2: FutureWarning: The provided callable <function median at 0x000001C92C87FE20> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  orders_products_final['median_days_since_prior_order'] = orders_products_final.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [57]:
#See results
orders_products_final.head()
# the "median_days_since_prior_order" was added

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,...,income,max_order_number,price_range_products,busiest_days,busiest_period_of_day,region,loyalty_flag,mean_price_per_user,type_of_spender,median_days_since_prior_order
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,40423,10,mid-range,Regular days,Average orders,South,New,6.367797,Low spender,20.5
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender,20.5
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender,20.5
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,40423,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender,20.5
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,40423,10,mid-range,Least busiest days,Most orders,South,New,6.367797,Low spender,20.5


In [58]:
#Evaluate "median_days_since_prior_order"
orders_products_final['median_days_since_prior_order'].describe()
#the median ranges from 0 to 30 days

count    3.099297e+07
mean     9.957106e+00
std      6.665469e+00
min      0.000000e+00
25%      6.000000e+00
50%      7.000000e+00
75%      1.250000e+01
max      3.000000e+01
Name: median_days_since_prior_order, dtype: float64

In [59]:
#Check aggregations
#Snap view of days_since_prior_order per user
orders_products_final.groupby('user_id').agg({'days_since_prior_order' : ['min', 'max', 'median']})
#nothing odd on the results

days_since_prior_order             
                           min   max median
user_id                                    
1                          0.0  30.0   20.5
2                          3.0  30.0   13.0
3                          7.0  21.0   10.0
4                          0.0  21.0   20.0
7                          2.0  30.0    9.0
...                        ...   ...    ...
206203                    27.0  30.0   30.0
206206                     0.0  15.0    3.0
206207                     1.0  30.0   16.0
206208                     0.0  20.0    7.0
206209                     3.0  30.0   22.0

[162633 rows x 3 columns]

## Conditions

In [60]:
#If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer”
orders_products_final.loc[orders_products_final['median_days_since_prior_order'] > 20, 'usage_frequency'] = 'Non-frequent customer'

C:\Users\Ich\AppData\Local\Temp\ipykernel_10992\1833112232.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Non-frequent customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  orders_products_final.loc[orders_products_final['median_days_since_prior_order'] > 20, 'usage_frequency'] = 'Non-frequent customer'


In [61]:
#If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer”
orders_products_final.loc[orders_products_final['median_days_since_prior_order'] <= 10, 'usage_frequency'] = 'Frequent customer'

In [62]:
#If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer”
orders_products_final.loc[(orders_products_final['median_days_since_prior_order'] > 10) & (orders_products_final['median_days_since_prior_order'] <= 20) , 'usage_frequency'] = 'Regular customer'

## Output evaluation

In [63]:
#Print the frequency of “usage_frequency” 
orders_products_final['usage_frequency'].value_counts(dropna = False) 

usage_frequency
Frequent customer        21222034
Regular customer          6873531
Non-frequent customer     2897401
Name: count, dtype: int64

In [68]:
#Check aggregations
#Snap view of usage_frequency variable
orders_products_final.groupby('usage_frequency').agg({'median_days_since_prior_order' : ['min', 'max', 'median']})
#nothing odd on the results

median_days_since_prior_order             
                                                min   max median
usage_frequency                                                 
Frequent customer                               0.0  10.0    7.0
Non-frequent customer                          20.5  30.0   28.0
Regular customer                               10.5  20.0   14.0

In [65]:
#See results
orders_products_final.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_sequence,reordered,product_name,...,max_order_number,price_range_products,busiest_days,busiest_period_of_day,region,loyalty_flag,mean_price_per_user,type_of_spender,median_days_since_prior_order,usage_frequency
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,10,mid-range,Regular days,Average orders,South,New,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,10,mid-range,Least busiest days,Average orders,South,New,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,10,mid-range,Least busiest days,Most orders,South,New,6.367797,Low spender,20.5,Non-frequent customer


Most of our users (68.38%) are 'Frequent customers' placing a follow-up order within 10 days of a previous order. 

# 10. Exporting data

In [69]:
#Check size before exporting
orders_products_final.shape
#9 new variables were added on the initially imported dataframe
#the number or rows were not altered

(30992966, 30)

In [70]:
#Exporting to prepared data folder
#The pickle format is preferred for large df. This df contains 31M rows
orders_products_final.to_pickle(os.path.join(path, '02 Data','Prepared data', 'orders_products_der_step6.pkl'))